In [1]:
import pandas as pd
df = pd.read_csv('prabowo_text.csv')
df.head()

,full_text
0,Anggota DPR Komisi II Komarudin WatoBun menila...
1,Senin sore (28/4) saya menghadiri undangan per...
2,Dulu LBP bgitu kejam ke Presiden @prabowo bahk...
3,Karena saya cinta Polri makanya saya kritik. U...
4,Setuju untuk mas Angga Raka Prabowo menjadi Ju...


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3701 entries, 0 to 3700
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   full_text  3701 non-null   object
dtypes: object(1)
memory usage: 29.0+ KB


In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch

model_name = "mdhugol/indonesia-bert-sentiment-classification"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

sentiment_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# Labelkan datanya
df['label'] = df['full_text'].apply(lambda x: sentiment_pipeline(x)[0]['label'])

df.head()


,full_text,label
0,Anggota DPR Komisi II Komarudin WatoBun menila...,LABEL_1
1,Senin sore (28/4) saya menghadiri undangan per...,LABEL_1
2,Dulu LBP bgitu kejam ke Presiden @prabowo bahk...,LABEL_2
3,Karena saya cinta Polri makanya saya kritik. U...,LABEL_1
4,Setuju untuk mas Angga Raka Prabowo menjadi Ju...,LABEL_0


In [6]:
label_mapping = {'LABEL_0': 'positive', 'LABEL_1': 'neutral', 'LABEL_2': 'negative'}
df['label'] = df['label'].map(label_mapping)
df.head()



,full_text,label
0,Anggota DPR Komisi II Komarudin WatoBun menila...,neutral
1,Senin sore (28/4) saya menghadiri undangan per...,neutral
2,Dulu LBP bgitu kejam ke Presiden @prabowo bahk...,negative
3,Karena saya cinta Polri makanya saya kritik. U...,neutral
4,Setuju untuk mas Angga Raka Prabowo menjadi Ju...,positive


In [7]:
df['label'].value_counts()

label
neutral     1971
negative    1450
positive     280
Name: count, dtype: int64

In [8]:
df.to_csv("dataset/prabowo_text_labeled.csv", index=False)